Import libraries and extensions

In [1]:
import tensorflow as tf

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)
print(tf.__version__)
print('Num GPUs available: ', len(tf.config.list_physical_devices('GPU')))

import numpy as np
import os
import pickle

from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

2.5.0
Num GPUs available:  0


Import the image datasets for this institution. Also import the most recent global model weights, which will be used to initialize the model.

In [4]:
global_weights_dir = r"C:\Documents\Global_Model\global_model_0X.h5"
data_dir = r'C:\Documents\Institution_X'
train_dir = os.path.join(data_dir, 'train_seq')
test_dir = os.path.join(data_dir, 'test_seq')

batch_size = 32
input_shape = (224, 224, 3)

# create a data generator
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(train_dir, class_mode='binary', batch_size=batch_size,target_size = (224, 224))
test_generator = test_datagen.flow_from_directory(test_dir, class_mode='binary', batch_size=batch_size,target_size=(224, 224))


Found 43374 images belonging to 2 classes.
Found 21361 images belonging to 2 classes.
Found 10790 images belonging to 2 classes.


Function to construct the VGG16 Convolutional Neural Network (CNN) with our layer alterations.

In [5]:
def create_model(input_shape):

    #imports the mobilenet model and discards the last 1000 neuron layer.
    base_model=VGG16(weights='imagenet',include_top=False,input_shape=input_shape) 

    #we add dense layers so that the model can learn more complex functions and classify for better results.
    x=base_model.output
    x=keras.layers.GlobalMaxPooling2D()(x)
    x=keras.layers.Dense(1024,activation='relu')(x)
    x=keras.layers.Dense(1024,activation='relu')(x)
    x=keras.layers.Dropout(0.25)(x)
    x=keras.layers.Dense(1024,activation='relu')(x) #dense layer 2
    x=keras.layers.Dense(512,activation='relu')(x) #dense layer 3
    preds=keras.layers.Dense(1,activation='sigmoid')(x) #final layer with sigmoid activation function
    
    model=keras.Model(inputs=base_model.input,outputs=preds)
    model.trainable = True

    # We want to set the first 20 layers of the network to be non-trainable
    for layer in model.layers[:16]:
        layer.trainable=False
    for layer in model.layers[16:]:
        layer.trainable=True

    return model

Function for tuning the parameters and initialize, fit, and compile the CNN model. The global model weights are loaded into the model after it is compiled.

In [6]:
def fit_model(model,base_weights_dir,train_history_dir):
    
    steps_per_epoch = int(len(train_generator))
    validation_steps = int(len(test_generator))

    checkpointer = keras.callbacks.ModelCheckpoint(base_weights_dir, monitor='val_accuracy',mode='max',save_best_only=True, save_weights_only=True)

    adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',keras.metrics.Recall(),keras.metrics.Precision()])
    
    #load the global model weights into the model
    model.load_weights(global_weights_dir, by_name=True)
    
    reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='max')
    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='max')

    tensorboard_callback = TensorBoard(log_dir=r'C:\Documents\Institution_X\iter_Y\experiment')
    
    history = model.fit_generator(
                train_generator,
                steps_per_epoch=steps_per_epoch,
                epochs = 100,
                validation_data=test_generator,
                validation_steps=validation_steps,
                verbose=1,
                callbacks=[checkpointer,reduce_lr_loss,earlyStopping,tensorboard_callback]

            )


    with open(train_history_dir, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    return history



Call the functions to construct and fit the model.

In [ ]:
#save directories
base_weights_dir = r'C:\Documents\Institution_X\iter_Y\weights_directory\instX.h5'
train_history_dir = r'C:\Documents\Institution_X\iter_Y\train_history_directory\instx.pickle'

model = create_model(input_shape)
history = fit_model(model,base_weights_dir,train_history_dir)